### First install required libraries

In [22]:
import numpy as np  # linear algebra
import pandas as pd  # data processing, CSV file I/O (e.g., pd.read_csv)
import matplotlib.pyplot as plt  # for visualization
import plotly.express as px  # for Mapbox scatter plot

from glob import glob  # Assemble a list of path names that match a pattern in glob.
from category_encoders import OneHotEncoder
from ipywidgets import Dropdown, FloatSlider, IntSlider, interact
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.metrics import mean_absolute_error
from sklearn.pipeline import make_pipeline
from sklearn.utils.validation import check_is_fitted

### Preview Data
I selected the files for training by combining CSV files 1 through 4 from the Mexico City real estate dataset, and used file 5 for testing.

In [12]:
files = glob("./dataset/mexico-city-real-estate-[0-4].csv")
files

['./dataset/mexico-city-real-estate-3.csv',
 './dataset/mexico-city-real-estate-2.csv',
 './dataset/mexico-city-real-estate-4.csv',
 './dataset/mexico-city-real-estate-1.csv']

Create wrangle function, reads the file into a pandas DataFrame, and returns the DataFrame

In [13]:
def wrangle(filepath):
    # read CSV file to Dataframe
    df = pd.read_csv(filepath)
    return df

Reads each CSV file into a DataFrame and concatenates them into a single DataFrame named "df"

In [14]:
frames = [wrangle(file) for file in files]
df = pd.concat([wrangle(file) for file in files], ignore_index=True) 
print(df.info())
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18512 entries, 0 to 18511
Data columns (total 16 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   operation                   18512 non-null  object 
 1   property_type               18512 non-null  object 
 2   place_with_parent_names     18512 non-null  object 
 3   lat-lon                     16569 non-null  object 
 4   price                       18187 non-null  float64
 5   currency                    18187 non-null  object 
 6   price_aprox_local_currency  18187 non-null  float64
 7   price_aprox_usd             18187 non-null  float64
 8   surface_total_in_m2         6888 non-null   float64
 9   surface_covered_in_m2       17773 non-null  float64
 10  price_usd_per_m2            4765 non-null   float64
 11  price_per_m2                17049 non-null  float64
 12  floor                       1229 non-null   float64
 13  rooms                       495

,operation,property_type,place_with_parent_names,lat-lon,price,currency,price_aprox_local_currency,price_aprox_usd,surface_total_in_m2,surface_covered_in_m2,price_usd_per_m2,price_per_m2,floor,rooms,expenses,properati_url
0,sell,apartment,|Gustavo A. Madero|Distrito Federal|México|,"19.5010872,-99.1230477",2715592.0,MXN,2691025.94,143074.99,0.0,NaN,NaN,NaN,NaN,NaN,NaN,http://lindavista-norte.properati.com.mx/qygs_...
1,sell,house,|Benito Juárez|Quintana Roo|México|,"21.161908,-86.8515279",5050000.0,MXN,5005502.90,266129.83,400.0,338.0,665.324575,14940.828400,2.0,NaN,NaN,http://cancun.properati.com.mx/d0mg_venta_casa...
2,sell,store,|Gustavo A. Madero|Distrito Federal|México|,NaN,9500000.0,MXN,9416292.89,500640.29,340.0,1050.0,1472.471441,9047.619048,NaN,NaN,NaN,http://gustavo-a-madero.properati.com.mx/i3tn_...
3,sell,house,|Benito Juárez|Quintana Roo|México|,"21.141616,-86.86109",810000.0,MXN,802672.48,42676.05,92.0,78.0,463.870109,10384.615380,NaN,NaN,NaN,http://cancun.properati.com.mx/qr5j_venta_casa...
4,sell,apartment,|Iztapalapa|Distrito Federal|México|,"19.3603443968,-99.089448452",500000.0,MXN,495476.82,26343.24,48.0,NaN,548.817500,NaN,NaN,2.0,NaN,http://san-pablo.properati.com.mx/9e3n_venta_d...


### Prepare Data

Training Data

In [15]:
# Build `wrangle` function
def wrangle(filepath):
    # read CSV file to Dataframe
    df = pd.read_csv(filepath)
    
    # Ensure that there are no leading or trailing spaces in column names
    df.columns = df.columns.str.strip()
     
    # House in "Quintana Roo" state
    mask_type = df["property_type"] == "house"
    mask_apt = df["place_with_parent_names"].str.contains("Quintana Roo")
    df = df[mask_type&mask_apt]

    # Remove outliers by 10% of "surface_covered_in_m2"
    low, high = df["surface_covered_in_m2"].quantile([0.1,0.9])
    mask_area = df["surface_covered_in_m2"].between(low, high)
    df = df[mask_area]

    # Split "lat-lon" columns
    df[["lat","lon"]] = df["lat-lon"].str.split(",",expand=True).astype(float)
    df.drop(columns="lat-lon", inplace=True)
  
    # Split neigborhood
    df["borough"] = df["place_with_parent_names"].str.split("|", expand=True)[1]
    df.drop(columns="place_with_parent_names", inplace=True)
    
    # Drop columns that are more than 50% null values.
    for i in df.columns[1:]:
        if df[i].isnull().sum()/len(df)> 0.5:
            df.drop(columns=i, inplace=True)
    
    # Drop low-and high-cardinality categorical variables
    df.drop(columns=["operation","property_type","currency","properati_url"], inplace = True)
    
    # Drop leaky columns for the target "price_aprox_usd" and for features
    df.drop(columns=["price","price_aprox_local_currency","price_usd_per_m2","price_per_m2","surface_total_in_m2"], inplace = True)
       
            
    return df

In [16]:
frames = [wrangle(file) for file in files]
df = pd.concat([wrangle(file) for file in files], ignore_index=True) 
print(df.info())
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1246 entries, 0 to 1245
Data columns (total 6 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   price_aprox_usd        1242 non-null   float64
 1   surface_covered_in_m2  1246 non-null   float64
 2   floor                  167 non-null    float64
 3   lat                    1117 non-null   float64
 4   lon                    1117 non-null   float64
 5   borough                1246 non-null   object 
dtypes: float64(5), object(1)
memory usage: 58.5+ KB
None


,price_aprox_usd,surface_covered_in_m2,floor,lat,lon,borough
0,266129.83,338.0,2.0,21.161908,-86.851528,Benito Juárez
1,42676.05,78.0,NaN,21.141616,-86.861090,Benito Juárez
2,267921.60,330.0,NaN,21.098381,-86.867206,Benito Juárez
3,197574.31,279.0,2.0,20.676573,-87.043918,Solidaridad
4,899000.00,420.0,2.0,20.617743,-87.085662,Solidaridad


Testing Data

In [17]:
X_test = pd.read_csv("./dataset/mexico-city-real-estate-5.csv", usecols=["surface_covered_in_m2","lat-lon","place_with_parent_names"])
X_test["borough"] = X_test["place_with_parent_names"].str.split("|", expand=True)[1]
X_test[["lat","lon"]] = X_test["lat-lon"].str.split(",", expand=True).astype(float)

# Drop columns
X_test.drop(columns=["lat-lon","place_with_parent_names"],inplace=True)
# X_test = X_test[["price_aprox_usd","surface_covered_in_m2","lat","lon","borough"]]
X_test = X_test[["surface_covered_in_m2","lat","lon","borough"]]
# X_test = X_test.dropna(subset=["price_aprox_usd"])

X_test.info()
X_test.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4628 entries, 0 to 4627
Data columns (total 4 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   surface_covered_in_m2  4442 non-null   float64
 1   lat                    4129 non-null   float64
 2   lon                    4129 non-null   float64
 3   borough                4628 non-null   object 
dtypes: float64(3), object(1)
memory usage: 144.8+ KB


,surface_covered_in_m2,lat,lon,borough
0,254.0,19.288128,-99.138385,Tlalpan
1,75.0,19.362690,-99.150565,Benito Juárez
2,92.0,19.399120,-99.141464,Benito Juárez
3,62.0,19.291345,-99.124312,Tlalpan
4,85.0,19.354987,-99.061709,Iztapalapa


!! It appears that I'm still having trouble dropping the 'floor' column, even though it has over 50% missing values. Let’s go through a few troubleshooting steps.

In [18]:
for col in df.columns:
    print(f"Column name: '{col}', Length: {len(col)}")

# Drop the "Floor" column if it is not part of the features to predict
try:
    df.drop(columns='floor', inplace=True)
    print("Column 'floor' dropped successfully.")
except KeyError as e:
    print(f"Error: {e}")

Column name: 'price_aprox_usd', Length: 15
Column name: 'surface_covered_in_m2', Length: 21
Column name: 'floor', Length: 5
Column name: 'lat', Length: 3
Column name: 'lon', Length: 3
Column name: 'borough', Length: 7
Column 'floor' dropped successfully.


Let’s see if it works or not.

In [19]:
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1246 entries, 0 to 1245
Data columns (total 5 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   price_aprox_usd        1242 non-null   float64
 1   surface_covered_in_m2  1246 non-null   float64
 2   lat                    1117 non-null   float64
 3   lon                    1117 non-null   float64
 4   borough                1246 non-null   object 
dtypes: float64(4), object(1)
memory usage: 48.8+ KB


,price_aprox_usd,surface_covered_in_m2,lat,lon,borough
0,266129.83,338.0,21.161908,-86.851528,Benito Juárez
1,42676.05,78.0,21.141616,-86.861090,Benito Juárez
2,267921.60,330.0,21.098381,-86.867206,Benito Juárez
3,197574.31,279.0,20.676573,-87.043918,Solidaridad
4,899000.00,420.0,20.617743,-87.085662,Solidaridad


Beautiful!!

### Explore

Shows the location of the houses in the dataset and represents their prices using color.

In [23]:
# Plot Mapbox location and price
fig = px.scatter_mapbox(
    df, #Our dataframe
    lat="lat",
    lon="lon",
    width=800,
    height=800,
    color="price_aprox_usd",
    hover_data=["price_aprox_usd"],
    title="Location of the apartments",
    labels={"price_aprox_usd":"Price (USD)"}
)

fig.update_layout(mapbox_style="open-street-map")

fig.show()

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

### Machine Learning : Build Model

Build a model to predict the price using the features "surface_covered_in_m2", "lat", "lon" and "borough".

In [25]:
# Define the target and features
target = "price_aprox_usd"
features = ["surface_covered_in_m2","lat","lon","borough"]

# Drop rows with NaN values in the target and feature columns
df = df.dropna(subset=[target] + features)

# Extract the target and features from the cleaned DataFrame
X_train = df[features]
y_train = df[target]

X_train.head()

,surface_covered_in_m2,lat,lon,borough
0,338.0,21.161908,-86.851528,Benito Juárez
1,78.0,21.141616,-86.861090,Benito Juárez
2,330.0,21.098381,-86.867206,Benito Juárez
3,279.0,20.676573,-87.043918,Solidaridad
4,420.0,20.617743,-87.085662,Solidaridad


The Mean Absolute Error Graphing is a good way to present data, but the value that indicates the average error in each prediction is also useful. This value is called the Mean Absolute Error (MAE), which represents the average magnitude of errors in each prediction. The closer the MAE is to 0, the better the model fits the data.

In [26]:
# Calculate the baseline mean absolute error for your model
y_mean = y_train.mean()
y_pred_baseline = [y_mean]*len(y_train)
baseline_mae = mean_absolute_error(y_train, y_pred_baseline)

print("Mean apt price:", y_mean)
print("Baseline MAE:", baseline_mae)

Mean apt price: 283583.9882389937
Baseline MAE: 216395.2634774845


### Iterate

impute the missing entries in a row or column of a DataFrame

In [27]:
# Build Model
model = make_pipeline(
    OneHotEncoder(use_cat_names=True),
    SimpleImputer(),
    Ridge()
)
# Fit model
model.fit(X_train, y_train)

/workspaces/Predicting-House-Prices/.conda/lib/python3.11/site-packages/sklearn/base.py:411: FutureWarning:

The `_get_tags` method is deprecated in 1.6 and will be removed in 1.7. Please implement the `__sklearn_tags__` method.

/workspaces/Predicting-House-Prices/.conda/lib/python3.11/site-packages/sklearn/base.py:411: FutureWarning:

The `_get_tags` method is deprecated in 1.6 and will be removed in 1.7. Please implement the `__sklearn_tags__` method.



Pipeline(steps=[('onehotencoder',
                 OneHotEncoder(cols=['borough'], use_cat_names=True)),
                ('simpleimputer', SimpleImputer()), ('ridge', Ridge())])

### Evaluate

In [28]:
# Create a list of predictions for the observations in your feature matrix X_train
y_pred_training = model.predict(X_train)
mae_training = mean_absolute_error(y_train, y_pred_training)
print("Training MAE:", round(mae_training, 2))

print(X_test.info())
print(X_test.head())

Training MAE: 160287.13
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4628 entries, 0 to 4627
Data columns (total 4 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   surface_covered_in_m2  4442 non-null   float64
 1   lat                    4129 non-null   float64
 2   lon                    4129 non-null   float64
 3   borough                4628 non-null   object 
dtypes: float64(3), object(1)
memory usage: 144.8+ KB
None
   surface_covered_in_m2        lat        lon        borough
0                  254.0  19.288128 -99.138385        Tlalpan
1                   75.0  19.362690 -99.150565  Benito Juárez
2                   92.0  19.399120 -99.141464  Benito Juárez
3                   62.0  19.291345 -99.124312        Tlalpan
4                   85.0  19.354987 -99.061709     Iztapalapa


In [29]:
y_test_pred = pd.Series(model.predict(X_test))
y_test_pred.head()

0    1.167038e+06
1    9.971098e+05
2    1.020437e+06
3    8.215280e+05
4    8.475066e+05
dtype: float64

### Results

In [30]:
coefficients = model.named_steps["ridge"].coef_
features = model.named_steps["onehotencoder"].get_feature_names_out()
feat_imp = pd.Series(coefficients, index=features).sort_values(key=abs , ascending=True)
feat_imp

surface_covered_in_m2         1792.149832
lon                         -59275.484814
borough_Cozumel             128041.871180
borough_Benito Juárez       163653.249797
borough_Tulum               166243.002518
borough_Cancún              167257.648553
lat                        -181169.213309
borough_Solidaridad         214087.500068
borough_Playa del Carmen    249393.040051
borough_Bacalar            -271248.402047
borough_Othón P. Blanco    -340218.108331
borough_Chetumal           -477209.801790
dtype: float64

In [32]:
def make_prediction(area, lat, lon, borough):
    #1.input from user and make it to dictionaries
    data = {
        "surface_covered_in_m2": area,
        "lat": lat,
        "lon": lon,
        "borough": borough
    }
    
    #2.put dictionaries to dataframe
    df = pd.DataFrame(data, index=[0]) #create 1 row
    
    #3.feed to model for prediction
    prediction = model.predict(df).round(2)[0]
    return f"Predicted house price: ${prediction}"

In [33]:
interact(
    make_prediction,
    area=IntSlider(
        min=X_train["surface_covered_in_m2"].min(),
        max=X_train["surface_covered_in_m2"].max(),
        value=X_train["surface_covered_in_m2"].mean(),
    ),
    lat=FloatSlider(
        min=X_train["lat"].min(),
        max=X_train["lat"].max(),
        step=0.01,
        value=X_train["lat"].mean(),
    ),
    lon=FloatSlider(
        min=X_train["lon"].min(),
        max=X_train["lon"].max(),
        step=0.01,
        value=X_train["lon"].mean(),
    ),
    borough=Dropdown(options=sorted(X_train["borough"].unique())),
);

interactive(children=(IntSlider(value=236, description='area', max=600, min=57), FloatSlider(value=20.93858597…